This tutorial illustrates a few of the most common utility functions available in the `utils` directory.

# Time Conversions

Time conversion utilities exist between the four modes of GPS Week and time of week in seconds, GPS milliseconds since start of GPS Epoch, Unix milliseconds since start of unix epoch, and Python datetime objects (timezone assumed to be UTC if none provided).

Functionality exists for all 12 combinations between the four time instances, but here we show just one example of looping through each instance time.

In [ ]:
from datetime import datetime, timezone
import gnss_lib_py.utils.time_conversions as tc

Methods to convert `datetime` to the other types of time instances.

In [ ]:
time_now = datetime.now(tz=timezone.utc)
time_now

# convert datetime to GPS week and GPS time of week
gps_week, gps_tow = tc.datetime_to_tow(time_now)
print(f"gps week: {gps_week} gps tow: {gps_tow}")

# convert datetime to GPS milliseconds
gps_millis = tc.datetime_to_gps_millis(time_now)
print(f"GPS milliseconds: {gps_millis}")

# convert datetime to UNIX milliseconds
unix_millis = tc.datetime_to_unix_millis(time_now)
print(f"UNIX milliseconds: {unix_millis}")

Methods to convert GPS week and GPS time of week to the other types of time instances.

In [ ]:
# convert GPS week and GPS time of week to datetime
datetime = tc.tow_to_datetime(gps_week, gps_tow)
print("datetime in UTC: ",datetime.strftime("%d %B, %Y %H:%M:%S"))

# convert GPS week and GPS time to GPS milliseconds
gps_millis = tc.tow_to_gps_millis(gps_week, gps_tow)
print(f"GPS milliseconds: {gps_millis}")

# convert GPS week and GPS time to UNIX milliseconds
unix_millis = tc.tow_to_unix_millis(gps_week, gps_tow)
print(f"UNIX milliseconds: {unix_millis}")

Methods to convert GPS milliseconds to other types of time instances.

In [ ]:
# convert GPS milliseconds to datetime
datetime = tc.gps_millis_to_datetime(gps_millis)
print("datetime in UTC: ",datetime.strftime("%d %B, %Y %H:%M:%S"))

# convert GPS milliseconds to GPS week and GPS time
gps_week, gps_tow = tc.gps_millis_to_tow(gps_millis)
print(f"gps week: {gps_week} gps tow: {gps_tow}")

# convert GPS milliseconds to UNIX milliseconds
unix_millis = tc.gps_to_unix_millis(gps_millis)
print(f"UNIX milliseconds: {unix_millis}")

Methods to convert UNIX milliseconds to other types of time instances.

In [ ]:
# convert UNIX milliseconds to datetime
datetime = tc.unix_millis_to_datetime(unix_millis)
print("datetime in UTC: ",datetime.strftime("%d %B, %Y %H:%M:%S"))

# convert UNIX milliseconds to GPS week and GPS time
gps_week, gps_tow = tc.unix_millis_to_tow(unix_millis)
print(f"gps week: {gps_week} gps tow: {gps_tow}")

# convert GPS milliseconds to UNIX milliseconds
gps_millis = tc.unix_to_gps_millis(unix_millis)
print(f"GPS milliseconds: {gps_millis}")

# Visualizations

The `visualizations.py` file contains several plotting functionalities. We'll use some existing data to demonstrate their functionality.

**Note:** In this case, the example data is filtered to be seconds apart, in the regular
setting, such measurements would be removed. To prevent this from happening,
we set remove_timing_outliers to False here. For the full dataset, set this flag to True

In [ ]:
from gnss_lib_py.parsers.android import AndroidDerived2021

# load Android Google Challenge data
!wget https://raw.githubusercontent.com/Stanford-NavLab/gnss_lib_py/main/data/unit_test/android_2021/Pixel4XL_derived.csv --quiet -O "Pixel4XL_derived.csv"
derived_data = AndroidDerived2021("Pixel4XL_derived.csv", remove_timing_outliers=False)

`plot_metric` is the most basic plotting function to plot a row. You need to input the `navdata` object and what row(s) you want to plot. For single rows, it will plot against a default index.

In [ ]:
from gnss_lib_py.utils.visualizations import plot_metric

glonass_data = derived_data.where("gnss_id","glonass")
fig = plot_metric(glonass_data, "raw_pr_m", linestyle="None")

You can also use `plot_metric` to plot two rows against each other.

In [ ]:
fig = plot_metric(glonass_data, "iono_delay_m", "tropo_delay_m", linestyle="none")

For a more informative plot, you can also group by.

In [ ]:
fig = plot_metric(glonass_data, "iono_delay_m", "tropo_delay_m", groupby="sv_id")

By default, the visualization tools do not save to file, but you can save them to file by setting the `save` flag to `True`. Check out the documentatoin 

By default, the plotting functions save plots to the `results` folder, but you can add `save=False` to instead return the figure object.

`plot_metric_by_constellation` is similar to `plot_metric` but it breaks up the data into separate figures by constellation and signal type. You need values for rows called `sv_id` and `signal_type` to use this function. We can quickly see that the raw pseudorange value is much more informative when we can separate by constellation and satellite!

In [ ]:
from gnss_lib_py.utils.visualizations import plot_metric_by_constellation

fig = plot_metric_by_constellation(derived_data, "gps_millis", "raw_pr_m", save=False)

The `plot_skyplot` function plots the satellite skyplot using the satellite positions and estimate receiver position.

In [ ]:
from gnss_lib_py.algorithms.snapshot import solve_wls
from gnss_lib_py.utils.visualizations import plot_skyplot

state_estimate = solve_wls(derived_data)

fig = plot_skyplot(derived_data, state_estimate, save=False)